In [117]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [118]:
train_data_dir = "/Users/deepanshudubb/Documents/accident/training"
val_data_dir = "/Users/deepanshudubb/Documents/accident/validation"

In [119]:
img_width, img_height = 150, 150

In [120]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [121]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary', 
    shuffle=False 
)

Found 1840 images belonging to 2 classes.


In [122]:
validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 460 images belonging to 2 classes.


In [123]:
def extract_features(generator):
    features = []
    labels = []
    for _ in range(generator.samples // generator.batch_size):
        batch_x, batch_y = next(generator)
        features.append(batch_x.reshape(batch_x.shape[0], -1))
        labels.append(batch_y)
    return np.concatenate(features), np.concatenate(labels)

In [124]:
train_features_knn, train_labels_knn = extract_features(train_generator)
val_features_knn, val_labels_knn = extract_features(validation_generator)


In [125]:

knn = KNeighborsClassifier(n_neighbors=5) 
knn.fit(train_features_knn, train_labels_knn)

KNeighborsClassifier()

In [126]:
knn_predictions = knn.predict(val_features_knn)
knn_accuracy = accuracy_score(val_labels_knn, knn_predictions)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.6227678571428571


In [127]:
model_cnn = Sequential()

In [128]:
optimizer = Adam(learning_rate=0.0001)
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [131]:

model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model_cnn.add(BatchNormalization()) 
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(1, activation='sigmoid'))


optimizer = Adam(learning_rate=0.0001) 
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

epochs = 50

model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 22s 371ms/step - accuracy: 0.6482 - loss: 2.8231 - val_accuracy: 0.5134 - val_loss: 2.7411
Epoch 2/50
 1/57 ━━━━━━━━━━━━━━━━━━━━ 19s 346ms/step - accuracy: 0.8750 - loss: 0.6651

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8750 - loss: 0.6651 - val_accuracy: 0.5134 - val_loss: 2.7880
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 355ms/step - accuracy: 0.5771 - loss: 0.6627 - val_accuracy: 0.5134 - val_loss: 4.1748
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5000 - loss: 0.6866 - val_accuracy: 0.5134 - val_loss: 4.1849
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 365ms/step - accuracy: 0.6643 - loss: 0.6054 - val_accuracy: 0.5134 - val_loss: 4.0006
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.4062 - loss: 0.7241 - val_accuracy: 0.5134 - val_loss: 4.0578
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 366ms/step - accuracy: 0.7035 - loss: 0.5796 - val_accuracy: 0.5134 - val_loss: 3.5632
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.5000 - loss: 0.8562 - val_accuracy: 0.5134 - val_loss: 3.5395
Epoch 9/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 366ms/step - accuracy: 0.7050 - loss: 0.5681 - val_accuracy: 0.5312 - val_

In [132]:
loss, accuracy = model_cnn.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)

print("CNN Accuracy:", accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.7239 - loss: 0.6709
CNN Accuracy: 0.7589285969734192


In [85]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
import numpy as np

In [111]:

def predict_accident(image_path, model):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255. 

    prediction = model.predict(img)
    if prediction[0][0] > 0.5:
        return "Noxxn-Accidental"
    else:
        return "Accidental"

In [112]:
test_image_path = "/Users/deepanshudubb/Documents/accident/fine.jpg"
result_cnn = predict_accident(test_image_path, model_cnn)
print("CNN Prediction:", result_cnn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
CNN Prediction: Noxxn-Accidental


In [115]:
model_cnn.save("cnn_model.h5")

In [116]:
import pickle
with open("knn_model.pkl", "wb") as f:
    pickle.dump(knn, f) 